# LangChain을 이용한 답글 생성 자동화(여러 콘텐츠 생성 )


# LangChain을 활용한 리뷰 자동 생성 시스템, 큰 틀에서 이해하기

## 🎯 이 코드의 목적은?

여러 리뷰(예: 음식점 후기, 행사 평가 등)에 대해 **AI가 자동으로 정중한 답글**을 작성하게 만드는 것입니다.  
리뷰가 긍정적이든 부정적이든 **상황에 맞는 답변을 생성**하고, 필요하면 **분석도 해주는** 자동화 시스템을 만드는 것이죠.

---

## 🧠 전체 구조를 한눈에!

### 1. 📥 입력: 리뷰 데이터
- 사용자가 남긴 리뷰(ex. "음식이 맛있었어요", "직원이 불친절했어요")를 Excel 파일에서 불러옵니다.

### 2. 🛠️ LangChain으로 구성된 AI 시스템
LangChain이란? 쉽게 말해, **AI에게 질문하고 답을 받는 흐름을 '체인'처럼 만들 수 있게 도와주는 도구**입니다.

이 시스템은 크게 3가지 단계로 나눌 수 있습니다:

| 단계 | 하는 일 | 도구 |
|------|----------|------|
| **프롬프트 설계** | AI에게 정확한 역할과 질문을 알려주는 문장 템플릿 | `PromptTemplate` |
| **LLM 호출** | 실제로 OpenAI GPT에게 텍스트 생성 요청 | `ChatOpenAI` |
| **응답 처리** | 생성된 텍스트를 사람이 보기 좋게 출력 | `OutputParser` 등 |

---

## 🔁 주요 동작 흐름

1. **프롬프트 만들기**  
   → 리뷰 내용을 `{review}` 자리에 넣어줄 수 있는 틀을 만듭니다.

2. **LLM(GPT)에 전달**  
   → GPT에게 리뷰를 보여주고 “답변을 작성해줘”라고 요청합니다.

3. **응답 생성 후 출력**  
   → AI가 예의 바른 답글을 생성하고, 이를 사용자에게 보여줍니다.

---

## ✨ 추가 기능들

- **Few-Shot Prompting**:  
  AI에게 “이런 식으로 판단해줘”라는 예시를 몇 개 보여주고, 새로운 리뷰를 비슷한 방식으로 판단하게 합니다.  
  (예: 이 리뷰는 긍정인가? 부정인가?)

- **모델 성능 향상**:  
  GPT-3.5보다 똑똑한 GPT-4 또는 GPT-4o 모델을 사용할 수 있게 설정만 바꾸면 됩니다.

- **콘텐츠 생성 확장**:  
  영화 리뷰 작성, 특정 주제 설명(예: ‘AI’를 ‘물’에 비유) 등 다양한 형식의 콘텐츠도 자동으로 생성 가능하게 확장할 수 있습니다.

---

## 🧩 비유로 이해하기: AI 자동 답글 시스템은 마치...

> “AI 비서에게 손님 리뷰를 보여주고, 그에 맞게 정중하고 센스있게 답장 써달라고 시키는 것”과 같습니다.

- 우리는 리뷰(손님 편지)를 수집하고
- AI 비서에게 틀(프롬프트)을 제공하고
- 그 비서는 똑똑하게 판단해서, 상황에 맞는 멘트로 답장해주는 역할을 합니다.

---

## 📌 요약 정리

- 리뷰 자동 응답 = **프롬프트 + GPT + 출력처리**의 조합
- LangChain은 이 전체 흐름을 "파이프라인처럼" 잘 이어주도록 도와주는 도구
- 모델만 바꾸면 더 똑똑한 답변 가능 (GPT-3.5 → GPT-4o)
- 추가 응용: 영화 리뷰, 특정 개념 설명 등 콘텐츠 확장 가능!



# LangChain을 이용한 답글 생성 자동화

In [2]:
# 필요한 라이브러리 설치
# langchain: GPT 등 대형 언어모델을 연결하고 자동화할 수 있는 파이프라인 도구
# cohere, openai: 각각 LLM을 제공하는 회사들. 여기서는 openai를 사용
# tiktoken: 토큰 단위로 문자열 처리할 때 사용 (GPT의 길이 제한 등 체크용)
# numexpr, tabulate, llmdantic 등은 langchain 관련 하위 기능들에서 사용됨
!pip install langchain_community cohere openai tiktoken langchain numexpr tabulate llmdantic langchain_openai -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
# 데이터 처리를 위한 라이브러리
import pandas as pd

# LangChain에서 프롬프트 템플릿을 만들기 위한 도구
from langchain.prompts import PromptTemplate

# OpenAI의 LLM (GPT-3 기반 모델)을 LangChain에서 사용하는 모듈
from langchain.llms import OpenAI

# 운영체제 환경변수 설정 모듈 (API 키 등록용)
import os

# [포인트] GPT 모델을 사용하려면 OpenAI API 키가 필요합니다. 아래와 같이 등록합니다.
# ⚠️ API 키는 외부에 노출되면 안 되므로 실제 서비스에서는 환경변수나 비밀키 설정으로 관리합니다.
# os.environ["OPENAI_API_KEY"] = ""
# [포인트] 엑셀 파일로 된 리뷰 데이터를 불러옵니다.
# 예: 리뷰 내용이 담긴 'Review'라는 열이 존재한다고 가정
reviews = pd.read_excel('./sample_data/reviews.xlsx')


# 1단계: GPT에게 보낼 '프롬프트 틀' 만들기
# GPT는 질문을 어떻게 던지느냐에 따라 답이 달라지므로, 질문의 형식을 미리 템플릿으로 정합니다.
response_prompt = """
당신은 이공간의 관리자입니다.
다음 리뷰에 대한 답변을 작성해 주세요.
리뷰에 대한 공감과 친절한 인사를 포함해 주세요.
마지막으로 재방문을 기원하는 메시지를 남겨 주세요.
리뷰: {review}
메시지:
"""

# [포인트] LangChain의 PromptTemplate을 사용해 {review}라는 자리에 실제 리뷰 내용이 들어가도록 설정합니다.
prompt_template = PromptTemplate(input_variables=["review"], template=response_prompt)

# GPT 모델 불러오기 - OpenAI의 기본 LLM (기본은 text-davinci-003 계열)
# temperature는 창의성 조절 옵션 (0=정확하게, 1=창의적으로). 여기선 0.7로 적당히 자유롭게 설정
llm = OpenAI(temperature=0.7)


# 2단계: 위 템플릿과 모델을 이용해 실제 답변을 생성하는 함수
def generate_responses(reviews, prompt_template):
    responses = []
    for review in reviews:
        # 리뷰 내용을 템플릿에 넣어 실제 GPT에게 보낼 질문을 생성
        prompt = prompt_template.format(review=review)

        # GPT 모델에 질문을 던져 응답 받기
        response = llm(prompt)

        # 응답 내용을 리스트에 저장
        responses.append(response)
    return responses


# [포인트] 실제로 3개의 리뷰에 대해 자동 답변을 생성해봅니다.
# .to_list()로 리스트로 변환 → 첫 3개만 사용
responses = generate_responses(reviews['Review'][:3].to_list(), prompt_template)

# 생성된 답변을 출력해 봅니다.
for response in responses:
    print("답글:", response)

<ipython-input-4-71c41138b475>:38: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7)
<ipython-input-4-71c41138b475>:49: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)


답글: 안녕하세요, 탄소와 예술의 결합에 대한 리뷰를 남겨 주셔서 감사합니다. 저희 이공간에서는 더 나은 서비스를 제공하기 위해 항상 노력하고 있습니다. 다음에는 더 만족스러운 경험을 제공할 수 있도록 노력하겠습니다. 고객님의 소중한 의견을 반영하여 더 나은 공간을 만들어 나가겠습니다. 다시 한번 방문해 주시기를 기대합니다. 감사합니다.
답글: 안녕하세요. 이공간을 이용해주셔서 감사합니다.
리뷰를 남겨주신 것에 감사드리며, 이번 리뷰는 큰 도움이 되지 못했다는 점 죄송합니다.
앞으로 더 나은 서비스를 제공할 수 있도록 노력하겠습니다.
다시 한번 방문해주시길 기대하며, 좋은 하루 보내세요!
답글: 안녕하세요. 이공간을 찾아주셔서 감사합니다. 리뷰 남겨주셔서 감사드립니다. 아이들과 함께 오는 공간으로서 접근성이 부족한 점 죄송합니다. 더 나은 서비스를 제공하기 위해 노력하겠습니다. 재방문을 기원하며, 앞으로도 많은 관심 부탁드립니다. 감사합니다.


## Few-Shot Prompting

이 코드에서는 Few-Shot Prompting 기법을 사용하여 예시를 제공하고, 유사한 질문에 대한 답변을 생성합니다.
 Few-Shot Prompt Template은 몇 가지 예시를 통해 모델이 새로운 질문에 대해 정확하게 응답할 수 있도록 도와줍니다.

In [6]:
# ========================= Few-Shot Prompting 시작 =========================
# [개념 설명]
# Few-Shot Prompting은 GPT에게 "이런 식으로 답해줘!" 하고 몇 가지 예시를 먼저 보여준 후,
# 새로운 입력이 들어오면 그 패턴에 맞게 답변을 생성하게 하는 방법입니다.
# 예를 들어, AI가 리뷰를 긍정/부정으로 판단하게 하고 싶다면,
# 우리가 미리 분석해 놓은 리뷰 2~3개를 먼저 예시로 주는 것입니다.

from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# ----------------------------------------------------------------------
# 1. [예시 데이터 준비]
# GPT에게 먼저 보여줄 "질문-답변 쌍" 예제들입니다.
# 이 예제들을 통해 GPT는 "아~ 이런 방식으로 분석하면 되는구나!" 하고 학습하게 됩니다.
# ----------------------------------------------------------------------
examples = [
    {
        "question": "리뷰: '음식이 너무 맛있었어요! 서비스로 준 디저트가 최고였어요.'",
        "answer": """
1. 음식에 대한 의견은 긍정적인가요? 네.
2. 서비스에 대한 의견은 부정적인가요? 아니오.
3. 전반적인 의견은 긍정적인가요? 네.

따라서 최종 결론: 긍정적
""",
    },
    {
        "question": "리뷰: '대기 시간이 너무 길었고, 직원들도 불친절했어요.'",
        "answer": """
1. 음식에 대한 의견은 긍정적인가요? 언급 없음.
2. 서비스에 대한 의견은 부정적인가요? 네.
3. 전반적인 의견은 긍정적인가요? 아니오.

따라서 최종 결론: 부정적
""",
    },
]

# ----------------------------------------------------------------------
# 2. [예시 출력 형식 정의]
# 위의 예시(question-answer)를 GPT에게 어떤 형태로 보여줄지를 정하는 부분입니다.
# 여기선 "질문: ~ \n 답변: ~"의 형태로 정리합니다.
# ----------------------------------------------------------------------
example_template = PromptTemplate(
    input_variables=["question", "answer"],  # 사용할 변수 이름 정의
    template="질문: {question}\n{answer}"    # 실제로 템플릿이 출력되는 구조
)

# 첫 번째 예시가 실제로 어떤 형태로 만들어지는지 확인해 봅니다.
print(example_template.format(**examples[0]))


# ----------------------------------------------------------------------
# 3. [FewShotPromptTemplate 구성]
# 이제 본격적으로 GPT에게 전달할 전체 프롬프트를 구성합니다.
# 예시들을 먼저 보여주고, 그 다음 실제로 분석할 리뷰를 던지는 방식입니다.
# ----------------------------------------------------------------------
prompt_template = FewShotPromptTemplate(
    prefix="아래의 예시처럼 리뷰에서 긍정적 또는 부정적 요소를 판단하고, 최종 결론을 내리세요.",  # GPT에게 주는 설명
    examples=examples,                  # 위에서 만든 질문-답변 예시 2개 사용
    example_prompt=example_template,   # 예시를 어떤 형태로 보여줄지 설정
    suffix="리뷰: {input}",            # 마지막에 실제 입력 리뷰가 여기에 들어갑니다.
    input_variables=["input"]          # {input} 자리에는 새 리뷰가 들어올 예정
)

# ----------------------------------------------------------------------
# 4. [새 리뷰 분석 요청 생성]
# 이 프롬프트는 위 예시들과 함께 GPT에게 전달될 준비가 됩니다.
# ----------------------------------------------------------------------
new_review = "음식은 맛있었지만, 서비스가 너무 느렸어요"

# GPT에게 보낼 최종 메시지 포맷을 생성합니다.
# 이 결과를 출력해보면 GPT에게 어떤 식으로 질문이 들어가는지 구체적으로 알 수 있습니다.
formatted_prompt = prompt_template.format(input=new_review)

# 생성된 전체 프롬프트 출력 (GPT에게 실제로 요청하게 될 내용)
print(formatted_prompt)

# 실제 GPT 모델에게 요청할 경우에는 아래 줄의 주석을 해제해서 사용합니다.
# print(llm.invoke(formatted_prompt))


질문: 리뷰: '음식이 너무 맛있었어요! 서비스로 준 디저트가 최고였어요.'

1. 음식에 대한 의견은 긍정적인가요? 네.
2. 서비스에 대한 의견은 부정적인가요? 아니오.
3. 전반적인 의견은 긍정적인가요? 네.

따라서 최종 결론: 긍정적

아래의 예시처럼 리뷰에서 긍정적 또는 부정적 요소를 판단하고, 최종 결론을 내리세요.

질문: 리뷰: '음식이 너무 맛있었어요! 서비스로 준 디저트가 최고였어요.'

1. 음식에 대한 의견은 긍정적인가요? 네.
2. 서비스에 대한 의견은 부정적인가요? 아니오.
3. 전반적인 의견은 긍정적인가요? 네.

따라서 최종 결론: 긍정적


질문: 리뷰: '대기 시간이 너무 길었고, 직원들도 불친절했어요.'

1. 음식에 대한 의견은 긍정적인가요? 언급 없음.
2. 서비스에 대한 의견은 부정적인가요? 네.
3. 전반적인 의견은 긍정적인가요? 아니오.

따라서 최종 결론: 부정적


리뷰: 음식은 맛있었지만, 서비스가 너무 느렸어요


# LangChain을 이용한 답글 생성 자동화

In [7]:
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm=ChatOpenAI(temperature=0.5, model='gpt-3.5-turbo',max_tokens=1000)



In [8]:
# 답변 템플릿 설정
reply_template = ChatPromptTemplate.from_messages([
    # 'system' 메시지: 모델의 역할과 규칙을 설정합니다. 이 경우, 모델은 "행사 관계자"로서 리뷰에 답변을 작성합니다.
    ('system', '''당신은 행사 관계자입니다.
    행사 리뷰에 관한 리뷰에 대한 답변을 작성해 주세요.
    부정적 리뷰는 더욱 개선하겠다고 끝에 감사합니다 라고하고
    중립 리뷰는 감사하다고 표현해주고
    좋은 리뷰는 공감과 친절한 인사를 포함해 주세요.'''),

    # 'user' 메시지: 사용자가 입력한 리뷰를 모델에 전달합니다.
    ('user', '''{review}''')
])


In [9]:

# 리뷰에 대한 답변을 생성하는 체인 설정
# reply_chain은 'reply_template'을 사용하여 답변을 생성한 뒤, 그 답변을 문자열로 변환하는 과정을 거칩니다.
reply_chain = reply_template | llm | StrOutputParser()

# 첫 번째 리뷰에 대한 답변 생성 및 출력
reply_chain.invoke({'review': reviews['Review'][1]})

'안녕하세요. 소중한 의견 감사드립니다. 더 나은 서비스를 제공하기 위해 노력하겠습니다. 피드백 주셔서 감사합니다.'

In [10]:
# 여러 리뷰에 대한 답변 생성 및 출력
for i, review in enumerate(reviews['Review'][:5]):
    print(f"Review {i+1}: {review}")  # 각 리뷰를 출력
    reply = reply_chain.invoke({'review': review})  # 각 리뷰에 대해 답변을 생성
    print(f"Reply {i+1}: {reply}")  # 생성된 답변을 출력
    print("=" * 50)  # 구분선을 출력

Review 1: 탄소와 예술의 결합 정말 별로
Reply 1: 안녕하세요. 소중한 의견을 남겨주셔서 감사합니다. 
탄소와 예술의 결합이 별로라는 점에 대해 안타깝게 생각합니다. 
더 나은 서비스를 제공할 수 있도록 계속 노력하겠습니다. 감사합니다.
Review 2: 크게 볼 것은 없습니다
Reply 2: 안녕하세요. 소중한 피드백 감사드립니다. 보다 나은 서비스를 제공할 수 있도록 노력하겠습니다. 귀하의 소중한 의견에 감사드리며, 더 나은 행사를 위해 노력하겠습니다. 
Review 3: 접근성 별로임. 차 있는 사람들에게 추천.
아이들과 같이 오는 공간.
이팝나무가 진 타이밍에 가서 그런지 볼게 없었음.
작품 전시회가 있어서 본 거 말고는 별로
Reply 3: 안녕하세요! 소중한 리뷰를 남겨주셔서 감사합니다. 접근성 부분에 대한 피드백을 주셔서 죄송합니다. 더 나은 서비스를 제공할 수 있도록 노력하겠습니다. 또한, 아이들과 함께 오시는 분들을 위한 더 많은 콘텐츠를 고려해 보겠습니다. 이팝나무가 아쉬웠다는 점도 인지하고, 앞으로는 더 많은 관람 포인트를 제공할 수 있도록 노력하겠습니다. 지적해주신 점을 반영하여 더 나은 곳으로 발전할 수 있도록 노력하겠습니다. 감사합니다.
Review 4: 이팝나무도서관 예약하고 갔음.
추운데 관계자 없어서 밖에서 떨면서 어찌저찌 연락되어 문 열어주면서 미안하단 말도 없었음
5분쯤 관람했을까? 홍보동영상 찍는다고 찍혀도 괜찮냐 물어서 싫다니 뒷쪽 방 열어줌
그곳에서 50분쯤 책 읽는데 안끝나겠다 싶어서 나옴
전주에 볼거 먹을거 많은데 일부러 시간내서 간게 너무 화가났음
비추
Reply 4: 안녕하세요. 이팝나무도서관을 이용해 주셔서 진심으로 감사드립니다. 고객님의 소중한 의견을 접수하였고, 불편을 끼쳐드려 죄송합니다. 관계자가 없어서 불편을 느끼셨고, 문을 열어주는 과정에서 미안한 인사가 없었다는 점에 대해 깊이 반성하고 있습니다. 또한 홍보동영상 촬영과 관련하여 고객님의 의견을 존중해야 했던 점도 인지하고 있습니다. 더 좋

# 성능향상(모델변경)

In [11]:
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
llm=ChatOpenAI(temperature=0.5, model='gpt-4o', max_tokens=1000)
reply_template = ChatPromptTemplate.from_messages([
    ('system','''당신은 행사 관계자입니다.
행사 리뷰에 관한 리뷰에 대한 답변을 작성해 주세요.
부정적 리뷰는 더욱 개선하겠다고 하고
중립 리뷰는 감사하다고 표현해주고
좋은 리뷰는 공감과 친절한 인사를 포함해 주세요.'''),
    ('user','''{review}''')

])

In [12]:
reply_chain = reply_template | llm | StrOutputParser()
reply_chain.invoke({'review':reviews['Review'][1]})

'행사에 대한 소중한 의견 감사합니다. 방문하신 분들께 더 많은 볼거리와 즐길 거리를 제공할 수 있도록 노력하겠습니다. 앞으로도 더욱 발전하는 모습 보여드리겠습니다. 감사합니다.'

In [13]:
# 여러 리뷰에 대한 답변 생성 및 출력
for i, review in enumerate(reviews['Review'][:5]):
    print(f"Review {i+1}: {review}")
    reply = reply_chain.invoke({'review': review})
    print(f"Reply {i+1}: {reply}")
    print("="*50)

Review 1: 탄소와 예술의 결합 정말 별로
Reply 1: 안녕하세요. 행사에 대한 솔직한 의견 감사드립니다. 탄소와 예술의 결합이 기대에 미치지 못한 점에 대해 유감스럽게 생각합니다. 앞으로 더 나은 행사를 만들기 위해 귀하의 피드백을 반영하여 개선할 수 있도록 노력하겠습니다. 더 나은 경험을 제공할 수 있도록 최선을 다하겠습니다. 감사합니다.
Review 2: 크게 볼 것은 없습니다
Reply 2: 안녕하세요, 행사에 참석해 주셔서 감사합니다. 기대에 미치지 못한 점이 있었다면 죄송합니다. 앞으로 더욱 흥미롭고 볼거리가 많은 행사를 만들기 위해 노력하겠습니다. 소중한 의견 감사드립니다.
Review 3: 접근성 별로임. 차 있는 사람들에게 추천.
아이들과 같이 오는 공간.
이팝나무가 진 타이밍에 가서 그런지 볼게 없었음.
작품 전시회가 있어서 본 거 말고는 별로
Reply 3: 안녕하세요,

귀중한 피드백에 감사드립니다. 접근성 문제에 대해 말씀해주셔서 감사드리며, 이를 개선하기 위해 더욱 노력하겠습니다. 또한, 방문 시 기대에 미치지 못한 점에 대해 아쉽게 생각합니다. 다음번에는 더욱 풍성하고 즐거운 경험을 제공할 수 있도록 최선을 다하겠습니다. 앞으로도 더 나은 행사를 위해 지속적으로 개선해 나가겠습니다. 감사합니다.
Review 4: 이팝나무도서관 예약하고 갔음.
추운데 관계자 없어서 밖에서 떨면서 어찌저찌 연락되어 문 열어주면서 미안하단 말도 없었음
5분쯤 관람했을까? 홍보동영상 찍는다고 찍혀도 괜찮냐 물어서 싫다니 뒷쪽 방 열어줌
그곳에서 50분쯤 책 읽는데 안끝나겠다 싶어서 나옴
전주에 볼거 먹을거 많은데 일부러 시간내서 간게 너무 화가났음
비추
Reply 4: 안녕하세요, 이팝나무도서관 행사 관계자입니다.

먼저, 도서관 방문 시 불편을 겪으신 점 진심으로 사과드립니다. 추운 날씨에 관계자가 제때 안내드리지 못해 불편을 드린 점과 이후 응대 과정에서 미흡했던 부분에 대해 사과의 말씀을 드립니다. 또한, 홍보 영상 촬영 관련하여 충분한 사전

## Langchain을 이용한 콘텐츠생성

## 영화리뷰 작성

In [14]:
# 이 코드는 LangChain Expression Language(LCEL)를 사용하여
# 입력된 영화에 대한 리뷰를 생성하고, 왜 그런 평가를 했는지 설명하는 예제입니다.
# 필요한 라이브러리 설치 및 API 키 설정
!pip install openai tiktoken langchain langchain_openai langchain_experimental -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 10.8 MB/s eta 0:00:00


In [17]:
# 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

# 프롬프트 템플릿 생성
template = ChatPromptTemplate.from_messages([
    ('user', """'{영화}'에 대한 리뷰를 작성하고,
이 영화가 왜 좋은지 영화를 안본 사람이 관심을 갖고 볼 수 있도록 잘 써주세요.
     이유도 한글로 상세히 작성해주세요.""")
])

# LLM 모델 설정
model = ChatOpenAI(temperature=0.9, model='gpt-4o', max_tokens=300)

# 체인 구성
review_chain = template | model | CommaSeparatedListOutputParser()

# 체인 실행 및 응답 생성
response = review_chain.invoke({'영화': '인셉션'})

# 결과 출력
print(response)

["'인셉션'은 크리스토퍼 놀란 감독이 만든 작품으로", '꿈과 현실', '그리고 인간의 무의식을 탐험하는 독창적인 스토리로 많은 이들의 찬사를 받은 영화입니다. 이 영화는 독특한 플롯과 놀라운 시각효과', '뛰어난 연기력을 통해 관객을 매료시킵니다.', '우선', "'인셉션'의 가장 큰 매력은 그 독창적인 스토리입니다. 영화는 꿈속의 꿈", '즉 다층적인 꿈의 세계를 배경으로 하고 있으며', '각 층의 꿈 속에서 벌어지는 사건들이 서로 연결되어 있습니다. 이러한 설정은 관객으로 하여금 영화 내내 긴장감을 늦출 수 없게 만들며', '한 순간도 눈을 뗄 수 없게 합니다.', '또한', '영화의 시각적 효과는 놀랍습니다. 꿈 속 세계를 시각적으로 구현하기 위해 사용된 특수효과와 촬영 기법은 현실과 환상을 넘나드는 느낌을 생생하게 전달합니다. 예를 들어', '꿈 속에서 중력이 뒤틀리거나 도시가 접히는 장면들은 관객에게 시각적 쾌감을 선사하며', '영화의 몰입감을 극대화시킵니다.', '이와 더불어', "영화에 출연한 배우들의 뛰어난 연기 역시 '인"]


## 특정 주제 설명

In [18]:
# LLM 체인 생성 및 파서 활용 예제
#이 코드는 LLM(대형 언어 모델) 체인을 생성하고, 주어진 프롬프트를 기반으로 LLM에서 결과를 출력하도록 구성하는 방법을 설명합니다. 또한, 출력 결과를 특정 형식으로 변환하는 파서(Parser) 사용 예제를 포함합니다. 이 예제는 Python을 사용하여 LangChain 라이브러리와 ChatGPT API를 연결하여 동작합니다.

## 1. 기본 체인 생성
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# 프롬프트 템플릿 설정
simple_template = ChatPromptTemplate.from_messages([
    ('system', "당신은 지식이 풍부한 AI입니다."),
    ('user', "{keyword}에 대해 {comparison}로 비유하여 설명해 주세요.")
])

# LLM 설정
llm = ChatOpenAI(temperature=0.5, model='gpt-4')

# 체인 생성
explanation_chain = LLMChain(prompt=simple_template, llm=llm)

# 체인 실행
response = explanation_chain.run({'keyword': 'AI', 'comparison': '물'})

print(response)


<ipython-input-18-0a8db2aca18b>:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.5, model='gpt-4')
<ipython-input-18-0a8db2aca18b>:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  explanation_chain = LLMChain(prompt=simple_template, llm=llm)
<ipython-input-18-0a8db2aca18b>:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = explanation_chain.run({'keyword': 'AI', 'comparison': '물'})


AI는 마치 물처럼 유연하고 변화무쌍합니다. 물은 그 어떤 형태의 용기에도 적응할 수 있으며, 때로는 강력한 파괴력을 가지기도 하고, 때로는 생명을 유지하는 데 필수적인 요소가 됩니다. 이와 같이 AI도 다양한 분야와 상황에 적응하며, 때로는 엄청난 효과를 가져다주고, 때로는 우리의 일상 생활에 필수적인 도구가 됩니다.

물이 계속 흐르듯, AI도 계속해서 학습하고 발전합니다. 물이 바위를 깎듯, AI는 방대한 양의 데이터를 처리하며 지식을 축적합니다. 또한 물이 여러 강을 통해 바다로 모이듯, AI는 여러 데이터 소스를 통합하여 보다 정확하고 풍부한 정보를 제공합니다.

하지만 물이 너무 많아지면 홍수를 일으키듯, AI도 잘못 사용되면 문제를 일으킬 수 있습니다. 따라서 AI의 사용은 적절한 관리와 규제가 필요합니다.

마지막으로, 물이 없으면 생명이 존재할 수 없듯이, AI도 우리의 미래 생활에서 빠질 수 없는 중요한 요소가 될 것입니다.
